<a href="https://colab.research.google.com/github/OmarMeriwani/Fake-Financial-News-Detection/blob/master/Dataset_Preparation_Company_Names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dataset Preparation - Company Names
This document contains the method of converting IEX company profiles to unique dataset to be used in fact checkin.

In [0]:
import pandas as pd
import nltk
import csv

Many company names hold other additions to the name such as Ltd or Inc. This method removes these additions to the nama.

In [0]:
def removetype(name):
    """
    Inc., Group Inc., Ltd., Holdings Inc., Worldwide Holdings Inc., L.P., Class A, Worldwide, Holdings, Holding, Corporation, Ltd, Inc, Industries
    """
    name = str(name).replace('Inc.','')
    name = name.replace('Ltd.','')
    name = name.replace(' Ltd ','')
    name = name.replace('Worldwide Holdings','')
    name = name.replace('L.P.','')
    name = name.replace('Holding','')
    name = name.replace('Corporation','')
    name = name.replace('Sponsored ADR','')
    name = name.replace(' ETF','')
    return name

Classes are part of the naming method of IEX companies. This method remove classes from the company name.

In [0]:
def getremoveclasses(name):
    tokens = nltk.tokenize.word_tokenize(name)
    for i in range(0, len(tokens)):
        try:
            if tokens[i] == 'Class':
                if tokens[i+1] in ['A','B','C','D','E','F','G','H','I']:
                    return ' '.join(tokens[:i]),'Class', tokens[i+1]
            if tokens[i] == 'Series':
                if str(tokens[i+1]) in 'ABCDEFGHIJKLMNOPQRSTUVWXYZ':
                    return ' '.join(tokens[:i]),'Series', tokens[i+1]
        except:
            continue


Processing the data of symbols.csv which has been downloaded from IEX API, the methods above are applied to each company name, then the word is tokenized and the count of each word is collected and stored.

In [0]:
df = pd.read_csv('symbols.csv')
namesList = []
previous = ''
count = 1
for i in range(0,len(df)):
    #,currency,date,exchange,iexId,isEnabled,name,region,symbol,type
    name = df.loc[i]['name']
    symbol = df.loc[i]['symbol']

    name = removetype(name)
    n = getremoveclasses(name)
    if n is not None:
        name = n[0]
    tokens = nltk.tokenize.word_tokenize(name)
    if len(tokens) >= 2:
        name = tokens[0]+ ' '+ tokens[1]
    else:
        name =tokens[0]
    if name != previous:
        previous = name
        count = 1
    else:
        count += 1

    namesList.append([name, symbol, count])


Only the names that have appeared one time is kept, many other additions to the names could not be removed, but finally there is a unique name for each company that could be collected.

In [0]:
namesWithMoreThanOne = [i for i,t,u in namesList if u > 1]
namesList = [i for i in namesList if i[0] not in namesWithMoreThanOne ]
for n in namesList:
    print(n)
df2 = pd.DataFrame(namesList)
df2.to_csv('UniqueCompanyNames.csv')